In [1]:
import yfinance as yf # yahoo finance library
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
def get_stock_data(STOCK_ID, start_time, end_time):
    stock_df = yf.download(STOCK_ID, 
                           start=start_time, 
                           end=end_time, 
                           progress=False)
    return stock_df

In [3]:
def get_start_and_end_dates(difference=1):
    
    start_date = datetime.now() 
    end_date = datetime.now() + relativedelta(days=difference)
    

    start_date = f'{start_date.year}-{start_date.month}-{start_date.day}'
    end_date = f'{end_date.year}-{end_date.month}-{end_date.day}'
    
    return start_date, end_date

In [4]:
STOCK_ID = '^DJI'    # DJIA (Dow Jones Industrial Average)

In [5]:
dates = get_start_and_end_dates()

In [6]:
# there are usually 2 records for the the current data nad the 2nd record should be selected

try:
    stock_data = get_stock_data(STOCK_ID, start_time = dates[0], end_time = dates[1]).iloc[1]
except:
    stock_data = get_stock_data(STOCK_ID, start_time = dates[0], end_time = dates[1]).iloc[0]

In [7]:
stock_data

Open         3.119801e+04
High         3.127222e+04
Low          3.112092e+04
Close        3.117601e+04
Adj Close    3.117601e+04
Volume       4.231550e+08
Name: 2021-01-21 00:00:00, dtype: float64